<a href="https://colab.research.google.com/github/eyyupcanisler/impark-internship/blob/main/impark_table_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl gspread gspread_dataframe gspread-formatting

import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from google.colab import drive
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
from gspread_formatting import *

# Mount Google Drive
drive.mount('/content/drive/')

# Authenticate user
auth.authenticate_user()

# Authorize Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)
holy_link = "insert_link"
# Load data from Google Sheets
sheet1 = gc.open_by_url(holy_link).sheet1
data1 = sheet1.get_all_values()
df1 = pd.DataFrame(data1[1:], columns=data1[0])

# Load additional data from Google Drive
df2 = pd.read_excel('connect_first_file_path')
df3 = pd.read_excel('connect_second_file_path')

In [ ]:
def check_date_format(date_str, date_format="%d.%m.%Y %H:%M"):
    try:
        pd.to_datetime(date_str, format=date_format)
        return True
    except ValueError:
        return False

def validate_and_update_dates(row):
    if not check_date_format(row['son_islem_tarihi']):
        return row['istek_tarihi']
    return row['son_islem_tarihi']

In [ ]:
df2['son_islem_tarihi'] = df2.apply(validate_and_update_dates, axis=1)

In [ ]:
# Change df2 column order
new_order = ["son_islem_tarihi", "paket_tipi", "il", "ilce", "bayi", "kurum", "durum", "plus_paket", "istek_tarihi"]
df2 = df2[new_order]

In [ ]:
# Adjust Modal_Info in df3
for index, row in df3.iterrows():
    if row['Modal_Info'] == 'Özdebir ile Eğitim Parkı':
        df3.at[index, 'Modal_Info'] = 'Özdebir ile Eğitim Parkı'
    elif row['Modal_Info'] == 'EP Bayi':
        df3.at[index, 'Modal_Info'] = row['Packet']

In [ ]:
df2["paket_tipi"] = df3["Modal_Info"]

In [ ]:
df4 = pd.concat([df2, df3], axis=1)
df4 = df4.drop(columns=['paket_tipi'])
df4 = df4.rename(columns={'Modal_Info': 'paket_tipi'})
df4 = df4.fillna('')

In [ ]:
# Filter out unwanted rows
unwanted_bayi = ["ep bayi", "EĞİTİMPARKİ BAYİ"]
unwanted_durum = ["Okul açıldı", "Paket yükseltme tamamlandı.", "Ek paket isteği"]

df4['bayi'] = df4['bayi'].astype(str)
pattern = '|'.join(unwanted_bayi)
df4 = df4[~df4['bayi'].str.contains(pattern, na=False)]
df4 = df4[~df4['durum'].isin(unwanted_durum)]

In [ ]:
# Map the columns
column_mapping = {
    'son_islem_tarihi': 'SİSTEM TARİHİ',
    'il': 'İL',
    'ilce': 'İLÇE',
    'bayi': 'İŞLER BAYİ-FRANCHİSE KURUMSAL ADI',
    'kurum': 'OKUL ADI',
    'paket_tipi': 'SATILAN ÜRÜN ADI',
    'Quota': 'SİPARİŞ KONTENJAN ADEDİ',
    'Phone_Number': 'OKUL İRTİBAT TELEFONU ',
    "Name" : "OKUL İRTİBAT KİŞİ",
    "Description": "AÇIKLAMALAR"
}

df4 = df4.rename(columns=column_mapping)

In [ ]:
# Convert dates to datetime for comparison
df1['SİSTEM TARİHİ'] = pd.to_datetime(df1['SİSTEM TARİHİ'], format="%d.%m.%Y %H:%M", errors='coerce')
df4['SİSTEM TARİHİ'] = pd.to_datetime(df4['SİSTEM TARİHİ'], format="%d.%m.%Y %H:%M", errors='coerce')

In [ ]:
# Ensure df1 has no duplicates based on 'OKUL ADI'
df1 = df1.drop_duplicates(subset=['OKUL ADI'])
df4 = df4.drop_duplicates(subset=['OKUL ADI'])

In [ ]:
# Perform the merge operation
merged_df = pd.merge(df1, df4, how='outer', on='OKUL ADI', suffixes=('', '_new'))

In [ ]:
# Update df1 with values from df4
for col in ['SİSTEM TARİHİ', 'İL', 'İLÇE', 'İŞLER BAYİ-FRANCHİSE KURUMSAL ADI', 'SATILAN ÜRÜN ADI', 'SİPARİŞ KONTENJAN ADEDİ', 'OKUL İRTİBAT TELEFONU ', 'OKUL İRTİBAT KİŞİ', 'AÇIKLAMALAR']:
    new_col = f'{col}_new'
    if new_col in merged_df.columns:
        merged_df[col] = merged_df.apply(
            lambda row: row[new_col] if pd.notna(row[new_col]) else row[col],
            axis=1
        )


In [ ]:
# Drop the columns with suffix '_new'
merged_df = merged_df.drop(columns=[col for col in merged_df.columns if col.endswith('_new')])

In [ ]:
# Drop the additional 'OKUL AÇILDI' and 'packet' columns
columns_to_drop = ['durum', 'plus_paket', 'istek_tarihi', 'Packet', ]
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

new_order1 = ['TARİH', 'AÇILDI', 'SATILAN ÜRÜN ADI', 'SİPARİŞ KONTENJAN ADEDİ',
          'İŞLER MUHASEBE LOGO - SİPARİŞ NUMARASI', 'MUHASEBE AÇIKLAMA',
          'SİPARİŞ ONAYLAYAN KİŞİ', 'İL', 'İLÇE',
          'İŞLER BAYİ-FRANCHİSE KURUMSAL ADI', 'OKUL TÜRÜ (DEVLET - ÖZEL - KURS)',
          'OKUL ADI', 'OKUL İRTİBAT KİŞİ', 'OKUL İRTİBAT TELEFONU ', 'EMAIL',
          'SORUMLU VEYA GÖRÜŞEN PERSONEL',
          'ÜST YÖNETİCİ', 'DİJİREAD-TERCİH ROBOTU',
          'ÖZEL WEB-MOBİL UYGULAMA BEDELİ', 'SUB DOMAIN', 'DOMAIN',
          'AÇIKLAMALAR', 'CEVAPSIZ ÇAĞRI SAYISI',
          'KAYIT İŞLEMLERİ ANLATILDI', 'Kaynak dağılımı yapıldı.',
          'iŞLEM TAMAMLANDI', 'EĞİTİM VEREN PERSONEL', 'Açıldı','SİSTEM TARİHİ']

merged_df = merged_df[new_order1]

In [ ]:
# Ensure the 'TARİH' column has correct date format and sort by it
merged_df['SİSTEM TARİHİ'] = pd.to_datetime(merged_df['SİSTEM TARİHİ'], format="%d.%m.%Y %H:%M", errors='coerce')
#merged_df = merged_df.sort_values(by='SİSTEM TARİHİ')

In [ ]:
ep_olcme = merged_df[merged_df['SATILAN ÜRÜN ADI'].isin(['EP ÖLÇME'])]

In [ ]:
# Filter out both 'Dijidemi ÖLÇME' and 'Dijidemi Ölçme'
#merged_df = merged_df[~merged_df['SATILAN ÜRÜN ADI'].isin(['EP ÖLÇME'])]

In [ ]:
# Get all data in the sheet
data = sheet1.get_all_values()

# Determine the range to format (assuming dates are in the first column)
date_range = f'AC1:AC{len(data)}'

# Define the date format
fmt = cellFormat(
    numberFormat=numberFormat(type='DATE_TIME', pattern='dd.MM.yyyy HH:mm')
)

# Apply the date format to the specified range
format_cell_range(sheet1, date_range, fmt)

print('Date format applied successfully')

In [ ]:
# Clear existing data (optional)
sheet1.clear()

In [ ]:
# Write the DataFrame to the Google Sheet
set_with_dataframe(sheet1, merged_df)

In [ ]:
sheet_olcme = gc.open_by_url(holy_link).worksheet("insert_worksheet_name")

In [ ]:
# Clear existing data (optional)
sheet_olcme.clear()

In [ ]:
# Write the DataFrame to the Google Sheet
set_with_dataframe(sheet_olcme, ep_olcme)